In [2]:
import openai
import itertools
import time
import os
import pandas as pd
import csv
from collections import defaultdict

<span style="font-size: 30px;">Read Data</span>

In [10]:
def extractFormattedConvo(index):
    convosPath = "../Dataset/ijcnlp_dailydialog/dialogues_text.txt"
    person = "Person 2: "
    formattedConvo =""    
    with open(convosPath,"r") as convosFile:
        allConvos = convosFile.readlines()
        splitConvo = allConvos[index].replace("__eou__\n","").split("__eou__")
        for message in splitConvo:
            if formattedConvo !="":
                formattedConvo +="\n"
            if(person == "Person 2: "):
                person = "Person 1: "
            else:
                person = "Person 2: "
            formattedConvo+=person+message
    return formattedConvo

In [9]:
def createDataframe():
    emotionsPath = "../Dataset/ijcnlp_dailydialog/dialogues_emotion.txt"
    dataFrame = pd.DataFrame({"index":[], "conversation":[], "emotions":[], "variety":[],"length":[]})
    try:
        with open(emotionsPath, "r") as file:
            #fetch line by line (one line is one convo)
            for index, emotions in enumerate(file):
                procEmotions = emotions.replace("\n","").replace(" ","")
                variety = len(set(procEmotions))
                length = len(procEmotions)
                convo = extractFormattedConvo(index)
                tempDF = pd.DataFrame({'index':[index], 'conversation':[convo], 'emotions': [procEmotions], 'variety': [variety], "length": [length]})
                dataFrame = pd.concat([dataFrame,tempDF], ignore_index=True)
    except Exception as e:
        print(f"The following exception occured: {e}")

    return dataFrame

In [11]:
dataFrame = createDataframe()

In [12]:
dataFrame.head()

,index,conversation,emotions,variety,length
0,0.0,Person 1: The kitchen stinks . \nPerson 2: I'...,20,2.0,2.0
1,1.0,"Person 1: So Dick , how about getting some cof...",42010,4.0,5.0
2,2.0,Person 1: Are things still going badly with yo...,0100,2.0,4.0
3,3.0,Person 1: Would you mind waiting a while ? \nP...,0004,2.0,4.0
4,4.0,Person 1: Are you going to the annual party ? ...,044,2.0,3.0


<span style="font-size: 30px;">Data Pre-Processing</span>

In [13]:
def countZeroes(string):
    return string.count('0')

In [14]:
def weighData(dataFrame, varietyWeight, zeroesWeight, lengthWeight):
    tempDataFrame = dataFrame.copy(deep=True)
    #normalize the data first
    min_value = tempDataFrame['variety'].min()
    max_value = tempDataFrame['variety'].max()
    tempDataFrame['variety'] = (tempDataFrame['variety'] - min_value) / (max_value - min_value)

    min_value = tempDataFrame['length'].min()
    max_value = tempDataFrame['length'].max()
    tempDataFrame['length'] = (tempDataFrame['length'] - min_value) / (max_value - min_value)

    tempDataFrame['zeroes'] = tempDataFrame['emotions'].apply(countZeroes)

    min_value = tempDataFrame['zeroes'].min()
    max_value = tempDataFrame['zeroes'].max()
    tempDataFrame['zeroes'] = (tempDataFrame['zeroes'] - min_value) / (max_value - min_value)

    #take weighted sum
    tempDataFrame['score'] = (varietyWeight * tempDataFrame['variety']) + (lengthWeight * tempDataFrame['length']) + (zeroesWeight * tempDataFrame['zeroes'])
    sortedTempDF = tempDataFrame.sort_values(by='score', ascending=False)
    return sortedTempDF[sortedTempDF['emotions'].str.len() > 5]


In [15]:
#dataFrame, varietyWeight, zeroesWeight, lengthWeight
weighedDF = weighData(dataFrame, 0.6,-1.2,0.4)
display(weighedDF.head())

,index,conversation,emotions,variety,length,zeroes,score
3769,3769.0,"Person 1: Jenny , you look terrible . What's e...",0105056505630,1.00,0.333333,0.15625,0.545833
6666,6666.0,Person 1: What ? How could you forget to reser...,601515,0.75,0.121212,0.03125,0.460985
9023,9023.0,"Person 1: Hello , three-five . \nPerson 2: Is...",00262525,0.75,0.181818,0.06250,0.447727
2687,2687.0,"Person 1: Brooke , I have bad news . \nPerson ...",0601515101510,0.75,0.333333,0.12500,0.433333
5990,5990.0,"Person 1: Tom , you look thoughtful and unhapp...",05650504,0.75,0.181818,0.09375,0.410227


<span style="font-size: 30px;">Evaluation</span>

In [16]:
def createCombos(convo,index,contextLength):
    message = convo[index]
    context = ""
    startingIndex = index
    endingIndex = 0
    if((index) - contextLength > 0):
        endingIndex = (index) - contextLength
    for i in range(startingIndex,endingIndex-1,-1):
        context = convo[i] + "\n" + context
    return message,context


In [17]:
import openai
import itertools
import time

def callGPT(messagesBox):
    openai.api_type = "azure"
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=messagesBox
        
    )
    extrResponse = response['choices'][0]['message']['content']

    if("NEUTRAL" in extrResponse.upper()):
        return "0"
    elif("ANGER" in extrResponse.upper()):
        return "1"
    elif("DISGUST" in extrResponse.upper()):
        return "2"
    elif("FEAR" in extrResponse.upper()):
        return "3"
    elif("HAPPINESS" in extrResponse.upper()):
        return "4"
    elif("SADNESS" in extrResponse.upper()):
        return "5"
    elif("SURPRISE" in extrResponse.upper()):
        return "6"
    else:
        return "7"

In [27]:
def createMB(message, context,filePath):

    prompt = "\n\nIdentify the emotion in the following message from the conversation above as either Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise:\n\n"
    
    sampleContext1 = "Person 1 : Hey! What is up?\nPerson 2 : Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?"
    sampleMessage1 = "Person 2 : Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?"
    samplePrompt1 =  sampleContext1 + prompt + sampleMessage1
    sampleAnswer1 = "Neutral"

    sampleContext2 = "Person 1: Let's go there!\nPerson 2: I am not going with you. just shut it!"
    sampleMessage2 = "Person 2: I am not going with you. just shut it!"
    samplePrompt2=  sampleContext2 + prompt + sampleMessage2
    sampleAnswer2 = "Anger"

    realPrompt = context + prompt + message

    
    messagesBox=""
    if('0' in filePath):
        #print("HEYY")
        messagesBox = [
            { "role": "user", "content": realPrompt}
        ]
    elif('1' in filePath):
        #print("Hey2")
        messagesBox = [
            { "role": "user", "content": samplePrompt1},
            { "role": "assistant", "content": sampleAnswer1},
            { "role": "user", "content": realPrompt}
        ]
    else:
        messagesBox = [
            { "role": "user", "content": samplePrompt1},
            { "role": "assistant", "content": sampleAnswer1},
            { "role": "user", "content": samplePrompt2},
            { "role": "assistant", "content": sampleAnswer2},
            { "role": "user", "content": realPrompt}
        ]

    return messagesBox

In [30]:
def evaluate(data,filePath):
    csv_file_path = filePath
    with open(csv_file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["ConvoIndex", "MessageIndex", "Context", "Response"])
        #for convo in data['conversation']:
        for index,row in data.iterrows():
            convo = row['conversation']
            ind = row ['index']
            convo = convo.split("\n")
            #print(convo)
            contextLength = 2
            for index,message in enumerate(convo):
                message, context = createCombos(convo,index,contextLength)
                messageBox = createMB(message,context,filePath)
                #call GPT
                try:
                    response = callGPT(messageBox)
                except Exception as e:
                    if('response was filtered due to the prompt triggering' in str(e)):
                        response = "8"
                    else:
                        print("Sleeping for 8 seconds.")
                        time.sleep(8)
                        try: 
                            response = callGPT(messageBox)
                        except Exception as e:
                            print("Exception:",str(e))
                            response="9"
                writer.writerow([ind, index, contextLength, response]) 
               


In [31]:
#2-shot-learning
weighedData = weighedDF.head(10).copy(deep=True)
filePath0 = "0shot.csv"
filePath1 = "1shot.csv"
filePath2 = "2shot.csv"

In [32]:
evaluate(weighedData,filePath0)

Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.


In [33]:
evaluate(weighedData,filePath1)

Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.


In [34]:
evaluate(weighedData,filePath2)

Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.
Sleeping for 8 seconds.


<span style="font-size: 30px;">Findings</span>

In [35]:
def calculate_accuracy(groundString, predString):
    if len(groundString) != len(predString):
        raise ValueError("Both strings must have the same length for accuracy comparison.")
    
    # Initialize a variable to count the number of matching characters
    match_count = 0
    exc = 0

    # Iterate through the characters of both strings
    for char1, char2 in zip(groundString, predString):
        if char1 == char2:
            match_count += 1
        elif char2 == "7" or char2 == "8" or char2 =="9":
            exc +=1


    # Calculate the accuracy as a percentage
    print("Number of messages:",len(groundString)-exc)
    print(exc," exception(s) occured.")
    accuracy = (match_count / (len(groundString)-exc)) * 100
    print("Accuracy: ", accuracy)

In [70]:
def calculate_precision_recall(ground_string, generated_string):
    # Create dictionaries to store true positives, false positives, and false negatives for each character in both strings
    tp = defaultdict(int)
    fp = defaultdict(int)
    fn = defaultdict(int)
    tn = defaultdict(int)
    categories = ["0","1","2","3","4","5","6"]

    for ground_char, generated_char in zip(ground_string, generated_string):
        if(generated_char == '7' or ground_char == '8' or ground_char == '9'):
            continue
        elif ground_char == generated_char:
            tp[ground_char] += 1
        else:
            fp[generated_char] += 1
            fn[ground_char] += 1
        for i in range(len(categories)):
            if(categories[i]!= ground_char and generated_char!=categories[i]):
                tn[categories[i]] +=1

    class_precision = {}
    class_recall = {}
    class_f1={}
    class_accuracy={}

    for char in set(ground_string):
        class_precision[char] = tp[char] / (tp[char] + fp[char]) if (tp[char] + fp[char]) > 0 else 0
        class_recall[char] = tp[char] / (tp[char] + fn[char]) if (tp[char] + fn[char]) > 0 else 0

        if class_precision[char] + class_recall[char] > 0:
            class_f1[char] = 2 * (class_precision[char] * class_recall[char]) / (class_precision[char] + class_recall[char])
        else:
            class_f1[char] = 0
        try:
            class_accuracy[char] = tn[char] + tp[char] / tn[char] + tp[char] + fn[char] + fp[char]
        except:
            class_accuracy[char] = 10000

    all_characters = sorted(set(ground_string))
    for char in all_characters:
        print(f"Class '{char}': Precision = {class_precision[char]:.2f}, Recall = {class_recall[char]:.2f}, F1 = {class_f1[char]:.2f}")

#precision = tp / (tp + fp)
#recall = tp / (tp + fn)

In [71]:
def findings(filePath):
    resultsDF = pd.read_csv(filePath)
    for i in set(resultsDF['Context']):
        results = resultsDF[resultsDF['Context']==i]
        predString = ""
        trueString =""
        for convoIndex in results['ConvoIndex'].drop_duplicates(keep='first'):
            predVal = results[results['ConvoIndex']==convoIndex]['Response'].to_string(index=False).replace("\n","").replace(" ","")
            trueVal = weighedData[weighedData['index']==convoIndex]['emotions'].to_string(index=False).replace("\n","").replace(" ","")
            predString += predVal 
            trueString += trueVal
        print("\n****Statistics - " + filePath + " learning ****\n")
        calculate_accuracy(trueString, predString)
        calculate_precision_recall(trueString, predString)



In [67]:
findings(filePath0)


****Statistics - 0shot.csv learning ****

Number of messages: 88
0  exception(s) occured.
Accuracy:  70.45454545454545
Class '0': Precision = 0.69, Recall = 0.60, F1 = 0.64
Class '1': Precision = 0.85, Recall = 0.85, F1 = 0.85
Class '2': Precision = 0.50, Recall = 0.33, F1 = 0.40
Class '3': Precision = 0.33, Recall = 0.50, F1 = 0.40
Class '4': Precision = 0.75, Recall = 0.75, F1 = 0.75
Class '5': Precision = 0.71, Recall = 0.79, F1 = 0.75
Class '6': Precision = 0.75, Recall = 0.82, F1 = 0.78


In [68]:
findings(filePath1)


****Statistics - 1shot.csv learning ****

Number of messages: 87
1  exception(s) occured.
Accuracy:  65.51724137931035
Class '0': Precision = 0.65, Recall = 0.52, F1 = 0.58
Class '1': Precision = 0.79, Recall = 0.85, F1 = 0.81
Class '2': Precision = 0.50, Recall = 0.33, F1 = 0.40
Class '3': Precision = 0.25, Recall = 0.50, F1 = 0.33
Class '4': Precision = 0.67, Recall = 0.75, F1 = 0.71
Class '5': Precision = 0.78, Recall = 0.74, F1 = 0.76
Class '6': Precision = 0.62, Recall = 0.73, F1 = 0.67


In [69]:
findings(filePath2)


****Statistics - 2shot.csv learning ****

Number of messages: 88
0  exception(s) occured.
Accuracy:  72.72727272727273
Class '0': Precision = 0.68, Recall = 0.70, F1 = 0.69
Class '1': Precision = 1.00, Recall = 0.77, F1 = 0.87
Class '2': Precision = 1.00, Recall = 0.33, F1 = 0.50
Class '3': Precision = 0.38, Recall = 0.75, F1 = 0.50
Class '4': Precision = 0.75, Recall = 0.75, F1 = 0.75
Class '5': Precision = 0.78, Recall = 0.74, F1 = 0.76
Class '6': Precision = 0.75, Recall = 0.82, F1 = 0.78
